In [1]:
import sys
sys.path.append("..")
from master import cds_utils as utils
# from master.cds_utils import *
import pandas as pd
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.preprocessing import MinMaxScaler

/Users/cindy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/cindy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
## Preparing Dataset

def preprocessing(dataframe_x,labels):
    list_of_x=[]
    list_of_y=np.zeros([1,1])

    for i in dataframe_x:
        list_of_x.append(np.append(0,i))
    
    list_of_y.fill(labels[2])
    return list_of_x,list_of_y

In [7]:
## The model

def model(X_train,y_train):
        # fix random seed for reproducibility
        np.random.seed(7)
        
        # reshape input to be [samples, time steps, features]
        X_train = np.reshape(X_train, (1, 60, 17))

        # create the model
        model = Sequential()
        model.add(LSTM(100))
        ## each 60 outputs will have a dimension of 100
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=3, batch_size=60)
        #print(model.summary())
        
        # Final evaluation of the model
        scores = model.evaluate(X_train, y_train, verbose=0)
        
        #Just to let me know my model is working correctly
        print(y_train)
        print(scores)
        print("Accuracy: %.2f%%" % (scores[1]*100))
        prediction=model.predict(X_train)
        print(prediction)


In [ ]:
## Retrieval of Raw Data

# (1) financial dataframe
financial = pd.read_csv('../master/AAPL - financial.csv', parse_dates=[1])
financial = utils.interpolate_data(financial, method='zero')
# (2) price dataframe
price = pd.read_csv('../master/AAPL - price.csv', parse_dates=[0]) # path to price 
price.set_index('Date', inplace=True)
# (3) technical dataframe
technical = pd.read_csv('../master/AAPL - technical.csv', parse_dates=[0])
technical.set_index('Date', inplace=True)
# camel case column names for technical
for old_column in technical:
    new_column = ' '.join([word.title() for word in old_column.split('_')])
    technical.rename(columns={old_column:new_column}, inplace=True)

# Parse the dataframes into a DataGenerator
data_generator = utils.DataGenerator(next='day', financial=financial, price=price, technical=technical)
i=0
for x_train,y_train in data_generator:
#     print(x_train.to_string())
#     print(x_train.iloc[1])
#     print(y_train)
#     print(x_train.columns)
#     print(len(y_train))

    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    x_train=x_train.drop("Quarter",axis=1)
    x_train = scaler.fit_transform(x_train.values)
    x,y=preprocessing(x_train, y_train)

    model(x,y)
    i+=1
    if i>10:
        break


Min date: 2008-09-26 00:00:00, Max date: 2018-06-29 00:00:00
**********************************************************
Iteration 0
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 3s 3s/step - loss: 0.7009 - acc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 49ms/step - loss: 0.6277 - acc: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 48ms/step - loss: 0.5592 - acc: 1.0000
[[1.]]
[0.49396881461143494, 1.0]
Accuracy: 100.00%
[[0.6101998]]
**********************************************************
Iteration 1
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 3s 3s/step - loss: 0.6684 - acc: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 52ms/step - loss: 0.5960 - acc: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 49ms/step - loss: 0.5275 - acc: 1.0000
[[0.]]
[0.4610190987586975, 1.0]
Accuracy: 100.00%
[[0.36935937]]
***********************************

In [ ]:
## Movie Example

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print(X_train)
print(X_train.shape)
print(len(X_train))
print(y_train)
print(len(y_train))
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

/Users/cindy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/cindy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


[[   0    0    0 ...   19  178   32]
 [   0    0    0 ...   16  145   95]
 [   0    0    0 ...    7  129  113]
 ...
 [   0    0    0 ...    4 3586    2]
 [   0    0    0 ...   12    9   23]
 [   0    0    0 ...  204  131    9]]
(25000, 500)
25000
[1 0 0 ... 0 1 0]
25000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 168s 7ms/step - loss: 0.4790 - acc: 0.7694
Epoch 2/3
25000/25000 [=